In [1]:
from cortexlib.utils import file as futils
from cortexlib.mouse import CortexlabMouse
from cortexlib.images import CortexlabImages
from cortexlib.vgg19 import PreTrainedVGG19Model, PoolingMode
from cortexlib.utils.random import set_global_seed
from cortexlib.utils.logging import Logger
from sklearn.linear_model import RidgeCV
from sklearn.decomposition import PCA
from PIL import Image
import numpy as np
import os


logger = Logger()
set_global_seed()

In [2]:
MOUSE_ID = futils.get_mouse_id()
logger.info(f"This notebook is running for mouse {MOUSE_ID}")

19:15:44 | INFO     | ℹ️ This notebook is running for mouse m02_d3


In [3]:
mouse = CortexlabMouse(mouse_id=MOUSE_ID)

logger.progress(f"Computing null distributions for all neurons in mouse {mouse.id}")
null_srv_all_neurons = mouse.compute_null_all_neurons(n_shuffles=100)
logger.success(f"Null distributions computed")

real_srv_all_neurons = mouse.compute_real_srv_all_neurons()
reliable_neuron_indices = mouse.get_reliable_neuron_indices(
            null_srv_all_neurons, real_srv_all_neurons, percentile_threshold=99)
neural_responses_mean, neural_responses, _ = mouse.get_responses_for_reliable_neurons(reliable_neuron_indices, real_srv_all_neurons, num_neurons=500)

logger.info(f"Neural responses shape: {neural_responses.shape}")

19:15:44 | INFO     | ⏳ Computing null distributions for all neurons in mouse m02_d3...
19:16:17 | SUCCESS  | ✅ Null distributions computed!
19:16:17 | INFO     | ℹ️ Neural responses shape: (1585, 2, 500)


In [4]:
pca = PCA(100)
neural_data_pcs = pca.fit_transform(neural_responses_mean)
pc1_neural_data = neural_data_pcs[:, 0]

In [5]:
futils.save_filtered_neural_data(
    mouse_id=mouse.id,
    neural_responses=neural_responses,
    neural_responses_mean=neural_responses_mean)

19:16:18 | INFO     | ℹ️ Skipping save, file already exists at /Users/callummessiter/workspace/msc-neuro/research-project/analysis/mouse_m02_d3/_neural_data/neural_data_mouse_m02_d3.pt


In [6]:
vgg19 = PreTrainedVGG19Model(pooling_mode=PoolingMode.AVGPOOL)

settings = vgg19.get_image_settings()
images = CortexlabImages(
    size=settings['size'],
    channels=settings['channels'],
    normalise_mean=settings['mean'], 
    normalise_std=settings['std'],
    rescale_per_image=settings['rescale_per_image'],
)

logger.progress("Loading and preprocessing images shown to mouse")
image_dataset = images.load_images_shown_to_mouse(mouse.image_ids)
logger.success("Images processed")

/Users/callummessiter/workspace/msc-neuro/research-project/myenv/lib/python3.11/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/Users/callummessiter/workspace/msc-neuro/research-project/myenv/lib/python3.11/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG19_Weights.IMAGENET1K_V1`. You can also use `weights=VGG19_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
19:16:19 | INFO     | ⏳ Loading and preprocessing images shown to mouse...
19:16:34 | SUCCESS  | ✅ Images processed!


In [7]:
logger.progress("VGG19: extracting features from images shown to mouse")
vgg19_feats, labels = vgg19.extract_features_with_pooling(image_dataset)
logger.success("VGG19 features extracted")

for layer, feats in vgg19_feats.items():
    logger.info(f"{layer} feats shape: {tuple(feats.shape)}")

19:16:34 | INFO     | ⏳ VGG19: extracting features from images shown to mouse...


  0%|          | 0/100 [00:00<?, ?it/s]

19:20:16 | SUCCESS  | ✅ VGG19 features extracted!
19:20:16 | INFO     | ℹ️ conv2_2 feats shape: (1585, 128)
19:20:16 | INFO     | ℹ️ conv3_4 feats shape: (1585, 256)
19:20:16 | INFO     | ℹ️ conv4_4 feats shape: (1585, 512)
19:20:16 | INFO     | ℹ️ conv5_4 feats shape: (1585, 512)
19:20:16 | INFO     | ℹ️ fc2 feats shape: (1585, 4096)


In [8]:
futils.save_model_features(model=futils.Model.VGG19, mouse_id=mouse.id, features=vgg19_feats, labels=labels)

19:20:16 | INFO     | ℹ️ Skipping save, file already exists at /Users/callummessiter/workspace/msc-neuro/research-project/analysis/mouse_m02_d3/_model_features/vgg19_features_mouse_m02_d3.pt


In [9]:
def regressor(X, Y):
    alphas = np.logspace(1, 7, 20)
    ridge = RidgeCV(alphas=alphas, store_cv_results=True)
    ridge.fit(X, Y)
    return ridge

ridge_models = {layer: regressor(vgg19_feats[layer].numpy(), pc1_neural_data) for layer in vgg19_feats}

In [10]:
output_dir="synthetic_pc1_images"
os.makedirs(output_dir, exist_ok=True)

# img_size = vgg19.get_image_settings()['size']
img_size = (96, 96)

for layer_name, feats in vgg19_feats.items():
    ridge = ridge_models[layer_name]
    synthetic_img = vgg19.generate_synthetic_image(layer_name, ridge, l2_lam=0.001)
    
    img_uint8 = (synthetic_img * 255).astype(np.uint8)
    img_pil = Image.fromarray(img_uint8, mode='L')
    img_resized = img_pil.resize((img_size[1]*2, img_size[0]*2), resample=Image.BICUBIC)

    filename = f"synthetic_img_{layer_name}.png"
    filepath = os.path.join(output_dir, filename)
    img_resized.save(filepath)